In [1]:
import pandas as pd
import numpy as np

In [40]:
import seaborn as sns
import matplotlib.pyplot as mlt
%matplotlib inline

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [6]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [39]:
import pickle

<br />

**FilePaths**

In [7]:
raw_file_path = r'/home/alahira/Documents/Data science projects/Income_predictions/Datasets/raw/'
processed_file_path = r'/home/alahira/Documents/Data science projects/Income_predictions/Datasets/processed/'
model_path = r'/home/alahira/Documents/Data science projects/Income_predictions/Models/'

## DATA PREPROCESSING

In [8]:
#Create column labels to be passed to pandas dataframe
labels= ['age','work_class','fnlwgt','education','years_of_education','marital_status','occupation','relationship','race'
         ,'sex','capital_gain','capital_loss','hours_per_week','native_country','target']

#load dataset 
income = pd.read_csv(raw_file_path + 'adult.csv', names=labels)

In [9]:
## typecasting

income['work_class'] = income['work_class'].astype('category')
income['education'] = income['education'].astype('category')
income['marital_status'] = income['marital_status'].astype('category')
income['occupation'] = income['occupation'].astype('category')
income['relationship'] = income['relationship'].astype('category')
income['race'] = income['race'].astype('category')
income['sex'] = income['sex'].astype('category')
income['native_country'] = income['native_country'].astype('category')
income['target'] = income['target'].astype('category')

print('Typecasting operations completed successfullty')

Typecasting operations completed successfullty


In [32]:
##seperate numeric and categorical variables for handling

numeric_vars = income.select_dtypes(exclude = ('category','object'))
cat_vars = income.select_dtypes(include = ('category','object'))

In [17]:
cat_vars.drop(labels='target', inplace = True, axis = 1)
cat_vars.head(5)

/home/alahira/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,work_class,education,marital_status,occupation,relationship,race,sex,native_country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba


In [14]:
numeric_vars.head(5)

,age,fnlwgt,years_of_education,capital_gain,capital_loss,hours_per_week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40


<br />

**Handle Categorical Data**

In [18]:
dummy_vars = pd.get_dummies(data = cat_vars)

In [20]:
dummy_vars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Columns: 102 entries, work_class_ ? to native_country_ Yugoslavia
dtypes: uint8(102)
memory usage: 3.2 MB


<br />

**Handle Numeric Variables**

In [21]:
#instantiate standard scaler
scaler = StandardScaler()

In [30]:
scaled_numeric_vars = scaler.fit_transform(numeric_vars)

In [34]:
scaled_numeric_vars = pd.DataFrame(data = scaled_numeric_vars, columns= numeric_vars.columns)

In [36]:
scaled_numeric_vars.head(5)

,age,fnlwgt,years_of_education,capital_gain,capital_loss,hours_per_week
0,0.030671,-1.063611,1.134739,0.148453,-0.21666,-0.035429
1,0.837109,-1.008707,1.134739,-0.145920,-0.21666,-2.222153
2,-0.042642,0.245079,-0.420060,-0.145920,-0.21666,-0.035429
3,1.057047,0.425801,-1.197459,-0.145920,-0.21666,-0.035429
4,-0.775768,1.408176,1.134739,-0.145920,-0.21666,-0.035429


In [ ]:
##binarize capital_gain and capital_loss variables

In [38]:
## merge dataframe
new_df = pd.merge(left=scaled_numeric_vars, right=dummy_vars, left_index=True, right_index=True)

In [ ]:
##remove outliers